<a href="https://colab.research.google.com/github/dscho15/DeepLearning/blob/main/Lecture1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 1

This lecture, we implement a KNN classifier to recognize images from a known dataset: MNIST.



In [4]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [7]:
import torch
from torchvision import datasets

mnist_train_dataset = datasets.MNIST('', train=True, download=True);
mnist_test_dataset = datasets.MNIST('', train=False, download=True); 

In [43]:
# The tensor sizes of the data labels etc.

train_labels = mnist_train_dataset.train_labels.numpy();
train_data = mnist_train_dataset.train_data.numpy().reshape((-1, 28*28));

test_labels = mnist_test_dataset.test_labels.numpy();
test_data = mnist_test_dataset.train_data.numpy().reshape((-1, 28*28))

print("The train labls and data: ", train_labels.shape, train_data.shape)
print("The test labels and data: ", test_labels.shape, test_data.shape)

The train labls and data:  (60000,) (60000, 784)
The test labels and data:  (10000,) (10000, 784)


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [52]:
cifar10_train_dataset = datasets.CIFAR10('../', train=True, download=True);
cifar10_test_dataset = datasets.CIFAR10('../', train=False, download=True);

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../cifar-10-python.tar.gz to ../
Files already downloaded and verified


In [54]:
cifar_train_labels = cifar10_train_dataset.labels.numpy()
cifar_train_data = cifar10_train_dataset.data.numpy()

cifar_test_labels = cifar10_train_dataset.test_labels.numpy()
cifar_test_data = cifar10_test_dataset.test_data.numpy()

AttributeError: ignored

In [51]:
import numpy as np

k = [1, 5, 20]
results = []

for i in k:

  # Make a classifier
  KNC = KNeighborsClassifier(i, algorithm='brute')

  # Fit the classifiers
  KNC.fit(train_data, train_labels) 

  # Get the results
  prediction = KNC.predict(test_data)

  # How well do we perform with k = 5?
  results.append(np.sum(prediction == test_labels)/test_data.shape[0])

# Get the results of this simple classifier, but do we need to parse all the data
print(results)

[0.9691, 0.9688, 0.9625]
